# Albumentations Integration for Anomaly Detection

In this notebook, we will explore **FiftyOne’s integration with Albumentations**, a powerful image augmentation library.
Augmentations can significantly enhance **anomaly detection** models by improving robustness and generalization.

![albumentation](https://cdn.voxel51.com/getting_started_manufacturing/notebook6/albumentation.webp)

## Learning Objectives:
- Understand the importance of **data augmentation** for anomaly detection.
- Use **Albumentations** to apply transformations to datasets in FiftyOne.
- Explore different augmentation techniques for improving model performance.

## Why Use Augmentations for Anomaly Detection?

Anomaly detection models often struggle due to **limited data availability** and **environmental variations**.
Data augmentation helps by:
- **Simulating real-world variations** (e.g., lighting changes, noise, blur).
- **Increasing dataset diversity**, reducing overfitting.
- **Improving model robustness** to unseen conditions.

Albumentations allows us to apply **realistic transformations**, such as:
- **Brightness and contrast adjustments** (simulate different lighting conditions).
- **Color jittering** (alter hue, saturation, and intensity).
- **Gaussian noise, blur, and distortions** (improve generalization).

**Relevant Documentation:**
- [Albumentations Documentation](https://albumentations.ai/docs/)
- [Why Data Augmentation Matters](https://docs.voxel51.com/integrations/albumentations.html)

In [ ]:
!pip install -U albumentations==1.4.24 fiftyone

In [3]:
!fiftyone plugins download https://github.com/jacobmarks/fiftyone-albumentations-plugin

/usr/local/lib/python3.12/dist-packages/glob2/fnmatch.py:141: SyntaxWarning: invalid escape sequence '\Z'
  return '(?ms)' + res + '\Z'

Copying plugin '@jacobmarks/albumentations_augmentation' to '/root/fiftyone/__plugins__/@jacobmarks/albumentations_augmentation'


In [ ]:
import gdown

# Download the coffee dataset from Google Drive

url = "https://drive.google.com/uc?id=1nAuFIyl2kM-TQXduSJ9Fe_ZEIVog4tth" # original
gdown.download(url, output="mvtec_ad.zip", quiet=False)

!unzip mvtec_ad.zip

import fiftyone as fo # base library and app
#import fiftyone.utils.huggingface as fouh # Hugging Face integration

dataset_name = "MVTec_AD"

# Check if the dataset exists
if dataset_name in fo.list_datasets():
    print(f"Dataset '{dataset_name}' exists. Loading...")
    dataset = fo.load_dataset(dataset_name)
else:
    print(f"Dataset '{dataset_name}' does not exist. Creating a new one...")
    # Clone the dataset with a new name and make it persistent
    #dataset_ = fouh.load_from_hub("Voxel51/mvtec-ad", persistent=True, overwrite=True)
    dataset_ = fo.Dataset.from_dir(
        dataset_dir="/content/mvtec-ad",
        dataset_type=fo.types.FiftyOneDataset
    )
    dataset = dataset_.clone("MVTec_AD")

In [2]:
# Try this for already loaded dataset
# dataset = fo.load_dataset('mvtec-ad-staging')
OBJECTS_LIST = [
    'pill',
    'zipper',
    'tile',
    'bottle',
    'transistor',
    'wood',
    'cable',
    'capsule',
    'carpet',
    'grid',
    'hazelnut',
    'leather',
    'metal_nut',
    'screw',
    'toothbrush'
]
OBJECT = "screw" ## object to select

In [3]:
from fiftyone import ViewField as F # helper for defining views

# Define the new dataset name for split set
dataset_name_split = "mvtec-screw"

if dataset_name_split in fo.list_datasets():
    print(f"Dataset '{dataset_name_split}' exists. Loading...")
    mvtec_screw = fo.load_dataset(dataset_name_split)
else:
    print(f"Dataset '{dataset_name_split}' does not exist. Creating a new one...")
    ## get the test split of the dataset
    test_split = dataset.match(F("category.label") == 'screw')

    # Clone the dataset into a new one called "mvtec_bottle"
    mvtec_screw = test_split.clone("mvtec-screw", persistent=True)

Dataset 'mvtec-screw' does not exist. Creating a new one...


In [4]:
print(mvtec_screw)

Name:        mvtec-screw
Media type:  image
Num samples: 480
Persistent:  True
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    category:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    defect:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    split:            fiftyone.core.fields.StringField
    defect_mask:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Segmentation)


In [ ]:
session = fo.launch_app(mvtec_screw, port=5156, auto=False)
print(session.url)

### How to manipulate Plugins

[Here](https://docs.voxel51.com/plugins/using_plugins.html#managing-plugins) is a guide you can follow to check your plugins, enable, disable or delete those.